# 对概念的理解和举例


### Character (字符)
- **定义**: 人类可读的文本单位
- **示例**: `'h'`, `'e'`, `'中'`, `'文'`, `'😀'`
- **Python 类型**: `str`
- **特点**: 
  - Unicode 字符数量巨大 (超过 100 万种)
  - 不同字符的 Unicode 码点不同
  - 字符数 ≠ 字节数

### Byte (字节)
- **定义**: 计算机存储的基本单位
- **范围**: 0-255 (2^8 = 256 种可能)
- **Python 类型**: `bytes`
- **特点**:
  - 固定范围: 0-255
  - 1 字节 = 8 位 (bit)
  - 所有文本都可以转换为字节序列

### Unicode
- **定义**：字符编码标准，为每个字符分配唯一的码点（标准）
- **Unicode 码点 (Code Point)**: 每个字符都有一个唯一的 Unicode 码点，通常用 `U+` 前缀表示（十六进制），例如: `U+0068`, `U+4E2D`, `U+1F600`
- **示例**: 
```python
ord('h')    # 104 (U+0068) 
ord('中')   # 20013 (U+4E2D)
```
- **Python 函数**: ord(char)


### UTF-8 编码

- **定义**: 将 Unicode 码点编码成字节序列的方式(计算机需要字节序列来存储)


- **示例**:
```python
'h'.encode('utf-8')    # b'h' (1 字节)
'中'.encode('utf-8')   # b'\xe4\xb8\xad' (3 字节)
'😀'.encode('utf-8')   # b'\xf0\x9f\x98\x80' (4 字节)
```

- **Python 方法**: char.encode('utf-8')

## 字符与字节的关系

```
Character (字符)
    ↓
Unicode 码点 (数字标识符)
    ↓
UTF-8 编码 (字节序列)
```

### 示例: `'中'`

```
字符: '中'
    ↓
Unicode 码点: 20013 (U+4E2D)
    ↓
UTF-8 编码: b'\xe4\xb8\xad' (3 字节)
    ↓
字节值: [228, 184, 173]
```

概念容易混淆：
- Unicode 不是字符，Unicode 是字符编码标准，为字符分配码点，是字符的数字表示。
- Unicode 和 UTF-8 不是一回事，Unicode 是标准，（定义字符和码点），UTF-8 是编码方式（将码点编码成字节）。



### 字节级别的优势

1. **初始词汇表大小固定**
   - 字符级别: 需要处理所有 Unicode 字符 (100万+)
   - 字节级别: 只需要 256 个初始 token (0-255)

2. **覆盖性**
   - 字符级别: 无法处理训练时未见过的字符
   - 字节级别: 可以处理任何 Unicode 字符

3. **统一性**
   - 字符级别: 不同语言的字符集差异很大
   - 字节级别: 所有文本都可以转换为字节序列

4. **简单性**
   - 字节范围固定 (0-255)
   - 可以学习常见字节组合的模式




# 分词器训练出现的概念


### 1. vocab (词汇表)

**类型**: `dict[int, bytes]`

**含义**: 从 token ID (整数) 映射到 token 的字节表示

**结构**:
```python
vocab = {
    0: b'\x00',        # token ID 0 -> 字节 0
    1: b'\x01',        # token ID 1 -> 字节 1
    ...
    104: b'h',         # token ID 104 -> 'h' 字节
    256: b'he',        # token ID 256 -> 'he' (BPE合并后的token)
    257: b'll',        # token ID 257 -> 'll' (BPE合并后的token)
    258: b'hello',     # token ID 258 -> 'hello' (BPE合并后的token)
}
```

**序列化后** (用于保存到 JSON):
```python
vocab_serialized = {
    'Ā': 0,           # 字节 0 对应的 unicode 字符
    'ā': 1,           # 字节 1 对应的 unicode 字符
    'Ġ': 32,          # 空格字节对应的 unicode 字符
    'h': 104,         # 'h' 字节
    'he': 256,        # 'he' token
    'll': 257,        # 'll' token
    'hello': 258,     # 'hello' token
}
```

---

### 2. merges (合并规则)

**类型**: `list[tuple[bytes, bytes]]`

**含义**: BPE训练过程中，按顺序记录的合并操作。每个元组 `(token1, token2)` 表示将 `token1` 和 `token2` 合并成一个新的 token。

**结构**:
```python
merges = [
    (b'h', b'e'),         # 第1次合并: h + e -> he
    (b'l', b'l'),         # 第2次合并: l + l -> ll
    (b'he', b'll'),       # 第3次合并: he + ll -> hell
    (b'hell', b'o'),      # 第4次合并: hell + o -> hello
]
```

**序列化后** (用于保存到文本文件):
```python
merges_serialized = [
    'h e',           # 第1次合并
    'l l',           # 第2次合并
    'he ll',         # 第3次合并
    'hell o',        # 第4次合并
]
```

---

### 3. 为什么需要序列化
序列化是指

JSON 不支持 `bytes` 类型！
---

## 🔄 BPE 合并过程示例

假设我们要对单词 **"hello"** 进行 BPE tokenization：

### 初始状态
```
原始文本: "hello"
UTF-8 编码: b'hello'
字节值: [104, 101, 108, 108, 111]
初始 tokens: [b'h', b'e', b'l', b'l', b'o']
```

### 应用 merges (按顺序)

**步骤 1**: 合并 `b'h'` + `b'e'`
```
合并前: [b'h', b'e', b'l', b'l', b'o']
合并后: [b'he', b'l', b'l', b'o']
```

**步骤 2**: 合并 `b'l'` + `b'l'`
```
合并前: [b'he', b'l', b'l', b'o']
合并后: [b'he', b'll', b'o']
```

**步骤 3**: 合并 `b'he'` + `b'll'`
```
合并前: [b'he', b'll', b'o']
合并后: [b'hell', b'o']
```

**步骤 4**: 合并 `b'hell'` + `b'o'`
```
合并前: [b'hell', b'o']
合并后: [b'hello']
```

### 最终结果
```
最终 tokens: [b'hello']
最终 token ID: 258 (假设 vocab[258] = b'hello')
```

---

## 💾 文件保存格式

### vocab.json
```json
{
  "Ā": 0,
  "ā": 1,
  "Ġ": 32,
  "h": 104,
  "he": 256,
  "ll": 257,
  "hello": 258
}
```

### merges.txt
```
h e
l l
he ll
hell o
```


# Profile 部分

step 1: add the special token to the vocabulary

In [63]:
vocab = {}

special_token = "<|endoftext|>"
vocab.update({special_token: 0})


step 2: add 256 byte values to the vocabulary

In [64]:
for i in range(256):
    char = chr(i)
    vocab.update({char: i + 1})

In [65]:
vocab

{'<|endoftext|>': 0,
 '\x00': 1,
 '\x01': 2,
 '\x02': 3,
 '\x03': 4,
 '\x04': 5,
 '\x05': 6,
 '\x06': 7,
 '\x07': 8,
 '\x08': 9,
 '\t': 10,
 '\n': 11,
 '\x0b': 12,
 '\x0c': 13,
 '\r': 14,
 '\x0e': 15,
 '\x0f': 16,
 '\x10': 17,
 '\x11': 18,
 '\x12': 19,
 '\x13': 20,
 '\x14': 21,
 '\x15': 22,
 '\x16': 23,
 '\x17': 24,
 '\x18': 25,
 '\x19': 26,
 '\x1a': 27,
 '\x1b': 28,
 '\x1c': 29,
 '\x1d': 30,
 '\x1e': 31,
 '\x1f': 32,
 ' ': 33,
 '!': 34,
 '"': 35,
 '#': 36,
 '$': 37,
 '%': 38,
 '&': 39,
 "'": 40,
 '(': 41,
 ')': 42,
 '*': 43,
 '+': 44,
 ',': 45,
 '-': 46,
 '.': 47,
 '/': 48,
 '0': 49,
 '1': 50,
 '2': 51,
 '3': 52,
 '4': 53,
 '5': 54,
 '6': 55,
 '7': 56,
 '8': 57,
 '9': 58,
 ':': 59,
 ';': 60,
 '<': 61,
 '=': 62,
 '>': 63,
 '?': 64,
 '@': 65,
 'A': 66,
 'B': 67,
 'C': 68,
 'D': 69,
 'E': 70,
 'F': 71,
 'G': 72,
 'H': 73,
 'I': 74,
 'J': 75,
 'K': 76,
 'L': 77,
 'M': 78,
 'N': 79,
 'O': 80,
 'P': 81,
 'Q': 82,
 'R': 83,
 'S': 84,
 'T': 85,
 'U': 86,
 'V': 87,
 'W': 88,
 'X': 89,
 'Y': 90

step 3: pretokenization

In [66]:
text = """low low low low low
lower lower widest widest widest
newest newest newest newest newest newest"""


words = text.split()
print(words)
from collections import Counter
word_counts = Counter(words)
frequency_table = dict(word_counts)

frequency_table = {tuple(word): count for word, count in frequency_table.items()}
frequency_table

['low', 'low', 'low', 'low', 'low', 'lower', 'lower', 'widest', 'widest', 'widest', 'newest', 'newest', 'newest', 'newest', 'newest', 'newest']


{('l', 'o', 'w'): 5,
 ('l', 'o', 'w', 'e', 'r'): 2,
 ('w', 'i', 'd', 'e', 's', 't'): 3,
 ('n', 'e', 'w', 'e', 's', 't'): 6}

step4: merge

In [42]:
#  count the frequency of all adjacent character pairs

merge_tables = {}
for word, count in frequency_table.items():
   
    for i in range(len(word)-1):
     
        char_pair = word[i:i+2]
        if char_pair not in merge_tables:
            merge_tables[char_pair] = count
        else:
            merge_tables[char_pair] += count
    
merge_tables


{('l', 'o'): 7,
 ('o', 'w'): 7,
 ('w', 'e'): 8,
 ('e', 'r'): 2,
 ('w', 'i'): 3,
 ('i', 'd'): 3,
 ('d', 'e'): 3,
 ('e', 's'): 9,
 ('s', 't'): 9,
 ('n', 'e'): 6,
 ('e', 'w'): 6}

In [47]:
# merge the most frequent pair
most_frequent_pair = max(merge_tables, key=merge_tables.get)
print("Most frequent pair:", most_frequent_pair, "with count:", merge_tables[most_frequent_pair])

# update the frequency table by merging the most frequent pair
new_frequency_table = {}
for word, count in frequency_table.items():
    new_word = []
    i = 0
    while i < len(word):
        if i < len(word) - 1 and word[i:i+2] == most_frequent_pair:
            new_word.append(''.join(most_frequent_pair))
            i += 2
        else:
            new_word.append(word[i])
            i += 1
    new_frequency_table[tuple(new_word)] = count


new_frequency_table

Most frequent pair: ('e', 's') with count: 9


{('l', 'o', 'w'): 5,
 ('l', 'o', 'w', 'e', 'r'): 2,
 ('w', 'i', 'd', 'es', 't'): 3,
 ('n', 'e', 'w', 'es', 't'): 6}

In [49]:
#  count the frequency of all adjacent character pairs
merge_tables = {}
for word, count in new_frequency_table.items():
    
     for i in range(len(word)-1):
      
          char_pair = word[i:i+2]
          if char_pair not in merge_tables:
                merge_tables[char_pair] = count
          else:
                merge_tables[char_pair] += count

merge_tables

{('l', 'o'): 7,
 ('o', 'w'): 7,
 ('w', 'e'): 2,
 ('e', 'r'): 2,
 ('w', 'i'): 3,
 ('i', 'd'): 3,
 ('d', 'es'): 3,
 ('es', 't'): 9,
 ('n', 'e'): 6,
 ('e', 'w'): 6,
 ('w', 'es'): 6}

In [50]:
# merge the most frequent pair
most_frequent_pair = max(merge_tables, key=merge_tables.get)
print("Most frequent pair:", most_frequent_pair, "with count:", merge_tables[most_frequent_pair])

# update the frequency table by merging the most frequent pair
frequency_table = {}
for word, count in new_frequency_table.items():
    new_word = []
    i = 0
    while i < len(word):
        if i < len(word) - 1 and word[i:i+2] == most_frequent_pair:
            new_word.append(''.join(most_frequent_pair))
            i += 2
        else:
            new_word.append(word[i])
            i += 1
    frequency_table[tuple(new_word)] = count


frequency_table

Most frequent pair: ('es', 't') with count: 9


{('l', 'o', 'w'): 5,
 ('l', 'o', 'w', 'e', 'r'): 2,
 ('w', 'i', 'd', 'est'): 3,
 ('n', 'e', 'w', 'est'): 6}

将merge这个过程写成loop

In [80]:
def merge(merge_counts):
    current_count = 1
    while current_count <= merge_counts:
        print("\nMerge iteration:", current_count)
        #  count the frequency of all adjacent character pairs
        merge_tables = {}
        for word, count in frequency_table.items():
            
             for i in range(len(word)-1):
              
                  char_pair = word[i:i+2]
                  if char_pair not in merge_tables:
                        merge_tables[char_pair] = count
                  else:
                        merge_tables[char_pair] += count

        # merge the most frequent pair
        most_frequent_pair = max(merge_tables, key=merge_tables.get)
        print("Most frequent pair:", most_frequent_pair, "with count:", merge_tables[most_frequent_pair])

        # update the frequency table by merging the most frequent pair
        new_frequency_table = {}
        for word, count in frequency_table.items():
            new_word = []
            i = 0
            while i < len(word):
                if i < len(word) - 1 and word[i:i+2] == most_frequent_pair:
                    new_word.append(''.join(most_frequent_pair))
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_frequency_table[tuple(new_word)] = count

        frequency_table.clear()
        frequency_table.update(new_frequency_table)
        current_count += 1 

    return frequency_table, merge_tables

In [81]:
frequency_table, merge_tables = merge(5)


Merge iteration: 1
Most frequent pair: ('lo', 'w') with count: 7

Merge iteration: 2
Most frequent pair: ('n', 'e') with count: 6

Merge iteration: 3
Most frequent pair: ('ne', 'w') with count: 6

Merge iteration: 4
Most frequent pair: ('new', 'est') with count: 6

Merge iteration: 5
Most frequent pair: ('w', 'i') with count: 3


In [82]:
merge_tables

{('low', 'e'): 2, ('e', 'r'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est'): 3}

In [83]:
frequency_table

{('low',): 5, ('low', 'e', 'r'): 2, ('wi', 'd', 'est'): 3, ('newest',): 6}